In [22]:
import datetime, requests

fecha_actual = datetime.date.today()
un_dia       = datetime.timedelta(days=1)

ANIO_INICIO = fecha_actual.year - 1
MES_INICIO  = fecha_actual.month
DIA_INICIO  = fecha_actual.day 

fecha_inicio = datetime.date(ANIO_INICIO, MES_INICIO, DIA_INICIO)

In [23]:
URL_BASE = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/' 

resultados = list()

f = open('datos.csv', 'a')

f.write('fecha,hora,temp_c,hum_per,pnm_hpa,dd_gr,ff_kmh,nombre\n')

while fecha_inicio < fecha_actual:
    r = requests.get(
        URL_BASE + 
        f'datohorario{fecha_inicio.year}{fecha_inicio.month:02}{fecha_inicio.day:02}.txt'
    )
    
    for line in r.iter_lines(decode_unicode='utf8'):
        if line.startswith((' ', 'F', 'o')):
            continue
        f.write(','.join(f'{line}\n'.split(maxsplit=7)))
            
    fecha_inicio += un_dia

f.close()

# TODO

In [ ]:
for line in r.iter_lines(decode_unicode='utf8'):
    if line.startswith(('F', ' ', 'o')):
        continue
    print(line)

In [ ]:
with open('datest.csv', 'w') as f:
    for line in r.iter_lines(decode_unicode='utf8'):
        if line.startswith(('F', ' ')):
            continue
        f.write(','.join(f'{line.strip()}\n').split(maxsplit=7))